# Import thư viện

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor
from tqdm.auto import tqdm
import requests
import re
import numpy as np
import pandas as pd
import queue
import time
import json

# Một số hàm cần thiết

In [2]:
# Save csv function
def save_csv(data:dict, path:str):
    df = pd.DataFrame(data)
    df.to_csv(path, index=False)
    
# Multiwork Function: giúp thực hiện nhiều function, công việc cùng lúc
def multiwork(func, inputs:list, number_of_workers: int):
    with ThreadPoolExecutor(max_workers=number_of_workers) as executor:
        results = list(executor.map(func, inputs))
    return results

In [3]:
# Get html with selenium Function: Hàm này lấy source html của một trang web dùng được cho các web sử dụng javascript để load data lên
def get_html_with_selenium(url):
    while True:
        try:
            # Khởi động trình duyệt Chrome với WebDriverManager
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
            driver.set_page_load_timeout(10)
            # Điều hướng đến trang web
            driver.get(url)

            # Lấy mã HTML của trang
            html_source = driver.page_source
            driver.quit()
            return html_source
        except:
            print(f"Can't get connection to {url}! Try again in 3 seconds.")
            time.sleep(3)

def get_scrolled_html_with_selenium(url):
    while True:
        try:
            # Khởi động trình duyệt Chrome với WebDriverManager
            driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
            driver.set_page_load_timeout(10)
            # Điều hướng đến trang web
            driver.get(url)
            
            # Lấy chiều cao ban đầu của trang
            last_height = driver.execute_script("return document.body.scrollHeight")

            while True:
                try:
                    # Cuộn xuống cuối trang
                    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                    
                    # Đợi một thời gian để nội dung mới được tải (thời gian này có thể thay đổi tuỳ theo tốc độ tải của trang)
                    time.sleep(2)
                    
                    # Lấy chiều cao mới sau khi cuộn
                    new_height = driver.execute_script("return document.body.scrollHeight")
                    
                    # Kiểm tra nếu không có nội dung mới tải nữa thì dừng cuộn
                    if new_height == last_height:
                        break
                    
                    # Cập nhật chiều cao mới để tiếp tục cuộn
                    last_height = new_height
                    
                except:
                    print(f"Can't get connection to {url}! Try again in 3 seconds.")
                    time.sleep(3)

            # Lấy mã HTML của trang
            html_source = driver.page_source
            driver.quit()
            return html_source
        except:
            print(f"Can't get connection to {url}! Try again in 3 seconds.")
            time.sleep(3)

In [4]:
def get_html(url):
    while True:
        # Gửi yêu cầu HTTP để lấy nội dung HTML
        response = requests.get(url)
        
        if response.status_code == 200:
            html_content = response.text
            return html_content
        
        print(f"Unable to connect to {url}! Try again in 3 seconds")
        time.sleep(3)

# eDoctor

In [15]:
def eDoctor_get_qa(url):
    html_content = get_html_with_selenium(url)
    
    # Lấy QA
    soup = BeautifulSoup(html_content, 'html.parser')
    elements = soup.select('.AnswerDetail_content__ZNHPp')
    try:
        question = str(elements[0].text)
        answer = str(elements[1].text)
    except:
        question = None
        answer = None
    
    # Lấy url
    related_url = []
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href.startswith("/hoi-dap/"):
            related_url.append(href)
            
    # Return
    return question, answer, related_url

In [16]:
keyword = "-da-lieu-"
url_seed = "https://edoctor.io/hoi-dap"
url_template = "https://edoctor.io{}"

data_dict = {
    "source": [],
    "url": [],
    "conversation": []
}

all_url = []
queue_url = queue.Queue()

In [ ]:
def eDoctor_crawling(check_point=5, batch_size=2):
    # Mở trang web chính và scroll để load các câu hỏi đáp
    html_content = get_scrolled_html_with_selenium(url_seed)
    # Lấy href các bài viết thêm vào hàng chờ nếu chưa được xử lý
    soup = BeautifulSoup(html_content, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href.startswith("/hoi-dap/") and url_template.format(href) not in all_url:
            all_url.append(url_template.format(href))
            queue_url.put(url_template.format(href))
            
    # Lấy QA từ các url trong hàng chờ
    process_urls = []
    while queue_url.empty() == False:
        batch_url = []
        while len(batch_url) < batch_size and queue_url.empty() == False:
            u = queue_url.get()
            if keyword in u:
                batch_url.append(u)
        process_urls.append(batch_url)
        
        if len(process_urls) == check_point or queue_url.empty():
            for process_url in tqdm(process_urls, desc=f"Crawling urls with batch_size={batch_size}: "):
                # Xử lý process_url
                results =  multiwork(eDoctor_get_qa, process_url, number_of_workers=5)
                for i, result in tqdm(enumerate(results), desc=f"Inserting {len(results)} QA: "):
                    question, answer, related_url = result
                    
                    # Thêm vào data_dict
                    data_dict["source"].append("eDoctor")
                    data_dict["url"].append(process_url[i])
                    data_dict["conversation"].append(json.dumps([
                        {"role": "Người dùng", "message": question},
                        {"role": "Bác sĩ", "message": answer}],ensure_ascii=False))
                    save_csv(data_dict, "eDoctor_data_crawling.csv")
                    
                    # Xử lý related_url
                    for href in related_url:
                        if url_template.format(href) not in all_url:
                            all_url.append(url_template.format(href))
                            queue_url.put(url_template.format(href))
                    
            # Reset
            process_urls = []
            
eDoctor_crawling()


# Ivie

In [5]:
url_seed = "https://ivie.vn/cong-dong"
url_page_template = "https://ivie.vn/cong-dong?page={}"
url_template = "https://ivie.vn{}"

data_dict = {
    "source": [],
    "url": [],
    "conversation": []
}

In [6]:
def Ivie_get_qa(url):
    try:
        html_content = get_html(url)
        soup = BeautifulSoup(html_content, 'html.parser')

        specialty = ""
        for e in soup.select('.specialist'):
            specialty += e.text + ", "
        specialty = specialty[:-2]
        
        if "Da Liễu" not in specialty:
            return None
        
        QA_detail = []
        conversation = []
        elements = soup.select('div.col.d-flex')
        for element in elements:
            QA_detail.append(element.text)
            
        elements = soup.select('.description')
        for i, element in enumerate(elements):
            t = element.text
            match = re.search(r'(Nữ|Nam), (\d+) tuổi(.*)', QA_detail[i], re.DOTALL) # Nếu match thì đây là câu hỏi
            if match and t in QA_detail[i]:
                conversation.append({"role": "Người dùng", "message": t})
            else:
                conversation.append({"role": "Bác sĩ", "message": t})
    
        return conversation
    except:
        print("Error with this QA!")
        return None
    
def Ivie_crawling_one_page(page_id):
    url = url_page_template.format(page_id)
    html_content = get_html_with_selenium(url)
        
    # Lấy các QA từ các href hợp lệ 
    url_qa = []
    soup = BeautifulSoup(html_content, 'html.parser')
    links = soup.find_all('a')
    for link in links:
        href = link.get('href')
        if href is not None:
            if "/cong-dong?id=" in href:
                url_qa.append(url_template.format(href))
    for u in url_qa:
        result_qa = Ivie_get_qa(u)
        if result_qa is not None:
            data_dict["source"].append("Ivie")
            data_dict["url"].append(u)
            data_dict["conversation"].append(result_qa)
    save_csv(data_dict, "Ivie_data_crawling.csv")

In [7]:
def Ivie_crawling():
    end = 0
    for i in tqdm(range(1, 4495), desc="Ivie Crawling 4494 pages: "):
        if i > end:
            if 4495 - end >= 100:
                end += 100
            else:
                end = 4495
            results = multiwork(Ivie_crawling_one_page, [j for j in range(i, end)], number_of_workers=10)
        else:
            continue

In [ ]:
Ivie_crawling()

# VNEXPRESS Da liễu

In [35]:
url_page_template = "https://vnexpress.net/suc-khoe/cac-benh/da-lieu/hoi-dap-p{}"
data_dict = {
    "source": [],
    "url": [],
    "conversation": []
}

In [36]:
def vnexpress_get_qa(html):
    try:
        # Phân tích cú pháp HTML bằng BeautifulSoup
        soup = BeautifulSoup(html, 'html.parser')

        # Tìm phần tử chứa câu hỏi
        question_div = soup.find('div', class_='content_less')
        if question_div:
            question_text = question_div.get_text(strip=True)  # Trích xuất nội dung câu hỏi
        else:
            return None

        # Tìm phần tử chứa câu trả lời của bác sĩ
        answer_div = soup.find('div', class_='fck_detail')
        if answer_div:
            # Xóa phần chứa tên và chức danh bác sĩ (thường trong thẻ div với class "name user_traloi")
            for name_tag in answer_div.find_all('div', class_='name user_traloi'):
                name_tag.decompose()  # Xóa phần tử khỏi nội dung
            
            answer_text = answer_div.get_text(strip=True)  # Trích xuất nội dung câu trả lời
        else:
            return None

        # Trả về kết quả
        return [{"role": "Người dùng", "message": question_text}, {"role": "Bác sĩ", "message": answer_text}]
    except:
        return None


In [37]:
def vnexpress_crawling():
    for i in tqdm(range(1,6), desc="VNEXPRESS Da liễu crawling 5 pages: "):
        url = url_page_template.format(i)
        html_content = get_html(url)
        soup = BeautifulSoup(html_content, 'html.parser')
        elements = soup.find_all("div",class_='list-answer')
        for element in elements:
            conversation = vnexpress_get_qa(str(element))
            if conversation is not None:
                data_dict["source"].append("VNEXPRESS")
                data_dict["url"].append(url)
                data_dict["conversation"].append(json.dumps(conversation,ensure_ascii=False))
        save_csv(data_dict, "Express_data_crawling.csv")
    print("DONE!")

In [ ]:
vnexpress_crawling()

# Bệnh viện 115

In [39]:
conversation = [[
    {
        "role": "Người dùng",
        "message": "Dạ em chào BS,\nEm xài son trước giờ không bị dị ứng, cách đây 3 ngày em tô son đã được 1 tiếng trước thì môi em nó hơi ngứa, và nó bị sưng.\nSon này hôm trước em có xài nhưng không sao ạ. Ba ngày em không thấy gì thay đổi, em nghĩ không sao nên hôm nay em tiếp tục tô son môi. Lúc xài tẩy trang ở môi thì môi em bị rát, nó tê tê, em sờ thì nó hơi sần.\nEm không biết là có phải bị dị ứng son không ạ?"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn,\nKhi bạn bị ngứa và sưng một vùng nào đó thì có thể bạn đã bị dị ứng, nhưng để xác định cụ thể bạn bị dị ứng do tác nhân nào thì rất khó.\nRiêng bạn vừa sử dụng son môi sau đó bị ngứa, sưng và có sự bong tróc... có thể là do tiếp xúc với son môi. Hiện tại bạn nên ngưng và dùng một loại son dưỡng ẩm không màu, không mùi cho môi."
    }],
    [
    {
        "role": "Người dùng",
        "message": "Chào BS,\nMặt tôi hiện tại đang bị mụn đỏ và mụn đầu đen kèm theo vết thâm sau mụn, da sần sùi, bị nám nhẹ và tàn nhang. Tôi thấy hiện tại có phương pháp lăn kim có thể cải thiện được làn da. Xin BS cho tôi hỏi với tình trạng da hiện tại của tôi có phù hợp dùng lăn kim không?\nNếu không thì có phương pháp nào điều trị thích hợp không? Chi phí điều trị khoảng bao nhiêu? Mong nhận được sự tư vấn của BS. Cảm ơn BS!"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn,\nLăn kim là một phương pháp làm tái tạo làn da khá tốt, vấn đề là thời điểm nào mình chọn lựa phương pháp nào cho phù hợp.\nHiện da bạn đang có mụn viêm, mụn đầu đen, bạn nên điều trị mụn trước. Sau khi tình trạng mụn ổn, bạn có thể tiếp tục điều trị sạm và tái tạo làn da."
    }],
    [
    {
        "role": "Người dùng",
        "message": "BS ơi giúp em với,\nNăm nay em 18 tuổi, bị mụn cũng 3 năm. Mụn trên mặt em không to, nó be bé như đầu kim mọc lung tung khắp mặt nhưng trên trán không có nên nhìn gần mới thấy.\nVới cả hiện tại da mặt lúc nào cũng đỏ đỏ và lúc rửa mặt xong có lúc ngứa.\nVà mặt em giờ nó mọc thêm vài cái nốt ruồi, bé thôi mà khá nhiều, khoảng 5 -6 cái, có cái to có cái nhỏ, bé màu nâu nhạt, to màu đen và không lồi trên bề mặt.\nEm đã tránh nắng rồi mà vẫn bị, mong BS giải đáp thắc mắc cho em, em cảm ơn."
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn,\nMụn thường xuyên nổi ở lứa tuổi dậy thì và kéo dài, điều cần quan tâm là ta phải biết chăm sóc để hạn chế sẹo mụn về sau.\nHiện em chưa trình bày rõ cho bác sĩ biết em đã và đang dùng thuốc gì thoa không và đã dùng bao lâu nên bác sĩ tạm hướng dẫn em: nếu em đang thoa kem gì thì em tạm ngưng kem đang thoa có thể da em đang bị kích ứng và em đến khám tại phòng khám chuyên khoa da liễu để bác sĩ thấy được sang thương như thế nào, từ đó em sẽ được tư vấn cụ thể hơn."
    }],
    [
    {
        "role": "Người dùng",
        "message": "Mông của em bị thâm, có mụn mọc nhiều như rôm, không màu không đau, một thời gian khô có vảy trắng. Mong BS tư vấn giúp em."
    },
    {
        "role": "Bác sĩ",
        "message": "Thân mến,\nTheo sinh lý thì mông của nhiều người cũng bị thâm, nhưng cần xác định rõ lại vùng nào của mông.\nHiện tại có “mụn mọc nhiều như rôm”, có ngứa hay không? Có lan rộng theo thời gian không? Những mụn này có trùng với lỗ chân lông không?... Bác sĩ khuyên bạn đến khám tại phòng khám chuyên khoa da liễu để xác định nhé."
    }],
    [
    {
        "role": "Người dùng",
        "message": "Chào BS,\nEm bị bệnh nấm da lâu năm không khỏi. BS có thể tư vấn giúp em phương pháp điều trị được không ạ? Em cảm ơn BS nhiều."
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn,\nNấm da là một loại bệnh dễ tái phát nếu bạn còn tiếp xúc với nguồn bệnh và môi trường sẽ làm cho nấm dễ phát triển.\nBạn bị bệnh nấm “đã lâu” và không biết bạn đã điều trị chưa? Nếu bạn đang sử dụng thuốc thì bạn tạm ngưng khoảng 01 tuần và đến khám tại phòng khám chuyên khoa da liễu để được khám xác định bệnh nấm và có thể cạo tìm loại nấm để có hướng điều trị."
    }],
    [
    {
        "role": "Người dùng",
        "message": "BS ơi,\nMình bị sẹo lồi ở cằm. Có tiêm 2 lần và đang bôi thuốc nhưng nó chưa xẹp nhiều. Vậy giờ mình phải làm sao ạ?"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn,\nSẹo lồi là do sự tăng sinh tại chỗ của nguyên bào sợi và sản xuất collagen gây phì đại, điều trị sẹo lồi khá khó khăn, chưa có phương pháp nào điều trị triệt để.\nCó nhiều phương pháp điều trị, bạn có thể tiếp tục tiêm sẹo lồi khoảng 4-6 tuần/lần hoặc áp lạnh bằng nitơ lỏng khoảng 1-2 tuần/lần. Hoặc bạn có thể phẫu thuật phối hợp với tiêm corticosteroid."
    }],
    [
    {
        "role": "Người dùng",
        "message": "Thưa BS,\nEm đi bắn tàn nhang bằng tia laser. Nhưng sau khi bắn xong da của em xuất hiện nhiều mảng nám và sạm da xuống, mà trước giờ da em không hề có. Vậy có cách nào giúp cho da hết vết sạm đó đi không ạ?"
    },
    {
        "role": "Bác sĩ",
        "message": "Tàn nhang là do sự tăng sắc tố của da.\nBạn cần xác định chính xác bạn có đúng là tàn nhang hay sạm da, hay đốm nâu.\nHiện điều trị những bệnh lý này cần phải phối hợp. Có thể dùng thuốc thoa, kem chống nắng, tia laser.\nCó nhiều loại laser, mỗi laser có một bước sóng khác nhau, sẽ phù hợp cho từng loại sang thương.\nLaser là phương pháp điều trị hiệu quả đối với tàn nhang nhưng bạn cần phải thực hiện đúng loại laser nào dùng cho điều trị tàn nhang.\nHiện tại nếu da bạn bị sạm, bạn có thể đến phòng khám chuyên khoa da liễu để BS xem sang thương hiện tại để đưa ra hướng điều trị thích hợp."
    }],
    [
    {
        "role": "Người dùng",
        "message": "Em chào BS,\nDa em thuộc da thường ít khi nổi mụn nhưng nổi thì để lại thâm rất lâu. Đã 3 tháng em dùng thuốc để bôi lên vết thâm, mặt nạ chanh tươi,... nhưng vết thâm mờ rồi lại đậm lên.\nEm không biết phải làm sao. Mong BS trả lời. Em cám ơn BS!"
    },
    {
        "role": "Bác sĩ",
        "message": "Thân mến,\nMụn là vấn đề rất thường gặp và vấn đề sẹo thâm sau mụn cũng vậy.\nKhi da em bị viêm sưng mụn và sau đó sẽ để lại thâm. Việc thoa thuốc, đắp mặt nạ chanh tươi cũng có tác dụng nhưng đôi khi không đều nên sẽ làm vết thâm lúc mờ, lúc đậm. Bạn nên đến phòng khám chuyên khoa da liễu để được tư vấn và có phương pháp điều trị hiệu quả hơn."
    }],
    [{ "role": "Người dùng", "message": "Thưa BS,\nEm xin được trình bày triệu chứng của bệnh mà em đang mắc phải và rất mong được BS tư vấn ạ!\nTừ năm 2017 trở về trước da mặt em bình thường. Em không dùng mỹ phẩm thường xuyên mà chỉ khi nào có công việc như lễ hội, tiệc thì em mới trang điểm.\nTuy nhiên, từ tháng 10/2017 đến nay da mặt của em ở vùng hai bên cằm, vùng trán, vùng thái dương nhìn vào bị sạm đen nhưng có lúc thì lại tím, đỏ. Từ khi da bị như thế em không trang điểm được và cũng không dùng được bất kỳ một loại mỹ phẩm nào.\nEm đã đi khám ở một số phòng khám nhưng mỗi phòng khám cho em một kết quả khác nhau; có nơi nói em bị viêm da, có nơi nói em bị tăng sắc tố da, có nơi nói em bị rối loạn nội tiết.\nTất cả các chẩn đoán trên đều không khỏi bệnh của em. Vì vậy hôm nay em nhờ BS tư vấn cho em nên đến BV nào để khám và chữa trị ạ? Em xin cảm ơn BS!" }, { "role": "Bác sĩ", "message": "Chào em,\nEm hiện đang có vùng “sạm” da kèm nhiều nơi “tím” lại “đỏ”, em chưa mô tả rõ mỗi vùng sạm/đỏ/tím đó như thế nào, em nên đến phòng khám chuyên khoa da liễu để BS chuyên khoa khám và đưa ra hướng điều trị." }],
    [{ "role": "Người dùng", "message": "Dạ chào BS ạ,\nTháng trước em có lăn kim điều trị rỗ ở 1 cơ sở nhỏ, và sau khi lăn xong đến giờ là 1 tháng rồi nhưng 2 bên má chỗ lăn kim của em rất đỏ và có dấu hiệu sậm màu.\nVậy giờ em phải làm sao ạ? Vết đỏ có thể bớt không và sẽ kéo dài bao lâu ạ? Hiện giờ em rất lo lắng, mong BS tư vấn giúp em ạ!" }, { "role": "Bác sĩ", "message": "Lăn kim là 1 biện pháp tái tạo da khá hiệu quả, nhưng thường lăn kim sau 1 tuần là da trở lại bình thường.\nHiện tại em lăn kim đã được 1 tháng nhưng vẫn còn đỏ và sạm, có thể em đã bị tăng sắc tố sau viêm. Trước hết em cần chăm sóc da và tránh nắng kỹ, dùng kem chống nắng và đến khám sớm tại phòng khám chuyên khoa da liễu để được hướng dẫn điều trị phục hồi." }],
    [{ "role": "Người dùng", "message": "Dạ em chào BS ạ,\nEm có một số thắc mắc về tình trạng da mặt của mình, xin nhờ BS tư vấn giúp em. Em 19 tuổi, hiện tại là sinh viên.\nChuyện là năm em học lớp 11, da của em bị mụn dị ứng nặng, hầu như cả mặt, là mụn viêm, rất đau, em bị trong thời gian gần 1 năm.\nEm có đi da liễu, chẩn đoán là mụn dị ứng, em sử dụng thuốc cả thoa và uống luôn nhưng đều không khỏi, nhiều người chỉ em rất nhiều loại mỹ phẩm, em đều sử dụng thử và đều không khỏi.\nLúc sau em vô tình được bà chị giới thiệu sử dụng kem hay gọi là kem con bướm đó ạ. Bỗng nhiên da mặt em dần dần hồi phục và hết sạch mụn, ban đầu còn thâm rất nhiều và dần đến hiện tại là mặt em không còn thâm luôn.\nTính đến hiện tại em xài kem này gần 2 năm, nhưng bây giờ em mới biết là em không ngưng sử dụng kem này hay đổi kem khác được luôn. Có vài lần em định bỏ, nhưng ngưng sử dụng vài ba hôm mặt em bị ngứa, nổi mẩn nỏ như rôm, da xấu lắm luôn, đến nỗi em không chịu được, phải mua về sử dụng tiếp.\nEm không biết là rốt cuộc da em hiện tại có vấn đề gì không và có thể bằng cách nào đó ngưng sử dụng kem này được không? Em nghe mọi người bảo là em cứ sử dụng mãi sẽ không tốt cho da, bảo bị quen kem gì đó, rồi lão hóa tàn nhang gì gì.\nEm cứ lo mãi mà cũng chẳng bỏ nó được. Em nhờ BS tư vấn giúp em ạ, em xin cảm ơn BS!" }, { "role": "Bác sĩ", "message": "Chào em,\nCó thể em đang sử dụng loại kem có thành phần không nên dùng lâu cho da, sử dụng lâu sẽ gây lệ thuộc và xuất hiện tác dụng phụ do thành phần đó gây nên. Do vậy, em rất khó bỏ loại kem này.\nEm cần ngưng sử dụng loại kem này và đến khám tại phòng khám da liễu để BS sẽ đưa ra một loại thuốc để hạn chế sự lệ thuộc thuốc và phục hồi da." }],
    [{ "role": "Người dùng", "message": "Em chào BS,\nEm tên Phước, 32 tuổi. Lúc trước em bị nám có điều trị ở phòng khám tư khoảng 2 năm và hoàn toàn khỏi bệnh vào khoảng giữa năm 2017.\nTừ đó đến nay em luôn uống thuốc chống nắng và đeo khẩu trang cẩn thận. Nhưng không hiểu sao da em càng ngày càng sạm. Không biết có phải khi không dùng thuốc nữa thì nám lại tái phát?\nBS tư vấn giúp em nhé! Em cảm ơn BS!" }, { "role": "Bác sĩ", "message": "Phước thân mến,\nĐúng như em đã mô tả, có thể em đang sử dụng loại kem có thành phần không nên dùng lâu cho da, sử dụng lâu sẽ gây lệ thuộc và xuất hiện sạm da.\nEm nên đến khám tại phòng khám da liễu để BS sẽ đưa ra hướng điều trị để hạn chế sự lệ thuộc thuốc và phục hồi da." }],
    [{
        "role": "Người dùng",
        "message": "Chào BS,\nBS cho tôi hỏi bàn chân của tôi có vết lỗ chỗ như trong ảnh, có mùi khó chịu. Xin hỏi nguyên nhân và cách chữa trị?"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn,\nHình bạn gửi BS nhìn chưa được rõ lắm nhưng BS có thể thấy em bị mụn cóc/mắt cá. Bạn nên đến chuyên khoa da liễu để được điều trị bằng laser hay áp lạnh bằng nitơ.\nVề mùi của chân, bạn không nói rõ là mùi xuất hiện khi nào nên chưa đủ thông tin để tư vấn thêm cho bạn."
    }],
    [{
        "role": "Người dùng",
        "message": "BS ơi,\nCả tuần nay cháu bị trong bóc da các đầu ngón tay 2 bàn tay đều bị ạ.\nBà nội cháu bảo rồi từ nó hết mà cả tuần chẳng thấy gì cả mà còn trong bóc hơn ạ. Nhất là mỗi khi tắm.\nGiờ phải làm sao để hết đây ạ? Cháu sợ bị gì lắm."
    },
    {
        "role": "Bác sĩ",
        "message": "Chào em,\nTheo mô tả và hình ảnh thì em bị viêm da tróc vảy. Em phải dùng kem giữ ẩm để hạn chế bong tróc/nứt da và sử dụng loại kem chuyên dành cho điều trị bệnh do BS da liễu kê cho em nhé."
    }],
    [{
        "role": "Người dùng",
        "message": "Cháu chào BS ạ,\nBS cho cháu hỏi cháu bị mụn ở ngực như trong hình 3 năm nay rồi, lúc đầu bé nhưng càng ngày càng lớn, không đau, động vào thấy hơi ngứa.\nBS cho cháu hỏi cháu bị sao ạ? Cháu xin cám ơn ạ!"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào bạn Mạnh,\nTheo hình bạn gửi thì tôi thấy đây là sẹo lồi.\nSẹo lồi là do sự tăng sinh tại chỗ của nguyên bào sợi và sản xuất collagen gây phì đại, điều trị sẹo lồi khá khó khăn, chưa có phương pháp nào điều trị triệt để.\nCó nhiều phương pháp điều trị, bạn có thể tiếp tục tiêm sẹo lồi khoảng 4-6 tuần/lần hoặc áp lạnh bằng nitơ lỏng khoảng 1-2 tuần/lần. Hoặc bạn có thể phẫu thuật phối hợp với tiêm corticosteroid."
    }],
    [{
        "role": "Người dùng",
        "message": "Bác sĩ cho em hỏi,\nEm bị phỏng và để lại sẹo như vậy. Không lồi cũng không lõm.\nEm muốn hỏi bệnh viện có xóa được sẹo này không ạ?"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào em Duyên,\nKhi phỏng sẽ làm tổn thương đến lớp bì sâu, do vậy làm ảnh hưởng đến sắc tố da. Nếu em muốn làm cho da đều màu thì cần đến BS chuyên khoa da liễu để được tư vấn biện pháp điều trị như: laser, thoa thuốc…"
    }],
    [{
        "role": "Người dùng",
        "message": "BS cho em hỏi,\nEm bị té xe hồi tết, giờ muốn da đều màu như trước thì phải làm sao ạ? Em cũng đang tiếp tục xức Nacurgo. Giờ nếu em dùng nghệ tươi hay kem nghệ thì có hiệu quả làm đều màu da không BS? Cảm ơn BS ạ!"
    },
    {
        "role": "Bác sĩ",
        "message": "Hình ảnh em gửi khá mờ nên chưa thấy rõ sẹo của em có lồi hay không và mức độ tổn thương da.\nThường sau khi bị té/chấn thương sẽ để lại vết tăng sắc tố khá lâu. Hiện tại vết sẹo đã lành nên em có thể ngưng dùng nghệ/kem nghệ.\nNếu em muốn làm cho da đều màu thì cần đến BS chuyên khoa da liễu để xác định mức độ tổn thương và tư vấn biện pháp điều trị."
    }],
    [{
        "role": "Người dùng",
        "message": "Thưa BS,\nEm bị bệnh chàm lâu năm, đã chữa nhiều phương pháp đông y tây y đều không khỏi. BS khám nói em bị chàm mãn tính.\nEm muốn hỏi về phương pháp điều trị bằng chiếu tia ánh sáng. Phương pháp này có hiệu quả không, phải thực hiện bao nhiêu lần, chi phí bao nhiêu và nên thực hiện ở BV nào? Mong BS tư vấn giúp em, cảm ơn BS rất nhiều!"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào em,\nChàm là dạng viêm da mạn tính (viêm da cơ địa), phương pháp điều trị chủ yếu là điều trị triệu chứng và hạn chế tái phát.\nHiện cũng có biện pháp điều trị bằng laser cho bệnh chàm, em có thể đến BV Da liễu."
    }],
    [{
        "role": "Người dùng",
        "message": "Thưa BS,\nEm bị bệnh chàm lâu năm, đã chữa nhiều phương pháp đông y tây y đều không khỏi. BS khám nói em bị chàm mãn tính.\nEm muốn hỏi về phương pháp điều trị bằng chiếu tia ánh sáng. Phương pháp này có hiệu quả không, phải thực hiện bao nhiêu lần, chi phí bao nhiêu và nên thực hiện ở BV nào? Mong BS tư vấn giúp em, cảm ơn BS rất nhiều!"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào em,\nChàm là dạng viêm da mạn tính (viêm da cơ địa), phương pháp điều trị chủ yếu là điều trị triệu chứng và hạn chế tái phát.\nHiện cũng có biện pháp điều trị bằng laser cho bệnh chàm, em có thể đến BV Da liễu."
    }],
    [{
        "role": "Người dùng",
        "message": "Xin chào BS,\nEm bị mụn trứng cá ung và mụn đầu đen trên mặt, vì em rất bận nên không thể đi khám được. BS có thể tư vấn cho em một số loại thuốc dùng tại nhà được không ạ?"
    },
    {
        "role": "Bác sĩ",
        "message": "Chào em,\nTheo mô tả, em hiện có nhiều mụn rất dễ để lại sẹo về sau. Em nên sắp xếp thời gian đến khám tại phòng khám chuyên khoa da liễu để có hướng điều trị và hướng dẫn thuốc thoa tại nhà cho em."
    }],
    [{
        "role": "Người dùng",
        "message": "Cháu chào BS,\nBS ơi, cháu bị sẹo do mụn nên cháu sử dụng thuốc trị sẹo Klirvin của Nga, sử dụng được 2 lần thì da mặt cháu nổi những mẩn đỏ như bị dị ứng thuốc.\nĐây là hình ảnh phần da cháu bị nổi mẩn đỏ! Có phải là bị dị ứng thuốc không ạ? Và nếu đúng thì biện pháp chữa trị như thế nào ạ? Cháu cảm ơn BS!"
    },
    {
        "role": "Bác sĩ",
        "message": "Nhung thân mến,\nDa em thuộc loại da nhờn và hiện tại bị nổi nhiều mụn viêm. Loại thuốc hiện tại em đang dùng không phù hợp để điều trị mụn. Em nên sắp xếp thời gian đến khám tại phòng khám chuyên khoa da liễu để có hướng điều trị và hướng dẫn thuốc thoa tại nhà cho em nhé."
    }],
    [{
        "role": "Người dùng",
        "message": "Cháu chào AloBacsi,\nNăm nay cháu 20 tuổi, cháu bị lây thủy đậu từ em. Cháu không phát hiện ra sớm nên mụn mọc rất nhiều ở mặt. Sau khi mụn vừa bong cháu liền bôi luôn kem nghệ Thái Dương.\nGiờ cháu bị đã gần 3 tuần, các vết thâm trên mặt không hết.\nBS cho cháu hỏi vết thâm sau này có hết không và có phải tại cháu bôi nghệ nên sẹo mới bị thâm không? Có cách nào trị thâm không ạ?\nBS tư vấn giúp cháu với ạ cháu lo quá. Cảm ơn BS ạ!"
    },
    {
        "role": "Bác sĩ",
        "message": "Hằng thân mến,\nHiện vết sẹo 3 tuần đã lành hẳn, em có thể ngưng kem nghệ, còn những vết tăng sắc tố từ từ sẽ hết trong vòng 3-6 tháng."
    }],
    [{
        "role": "Người dùng",
        "message": "Chào BS,\nEm tên Phúc, là nam và 23 tuổi. Cho em hỏi là:\n1/ Tại sao từ khi dậy thì tới giờ da mặt em tiết ra nhiều chất nhờn gây ra mụn và làm to lỗ chân lông rất nhiều, trong khi da tay và da các bộ phận khác lại rất ít tiết nhờn và không làm lỗ chân lông to nhiều, tại sao thế ạ?\nNếu biết được nguyên nhân có thể áp dụng các điều kiện trên da thông thường để điều trị cho da mặt được không? Em là người thuộc da dầu và da cũng trắng và mỏng nên rất dễ bị dị ứng nếu đi đường khói bụi nhiều.\n2/ Biện pháp để giảm thiểu chất nhờn tiết ra trên da mặt là gì? Vì em đọc nhiều bài viết cũng chỉ là giải pháp tạm thời.\nEm cảm ơn BS ạ."
    },
    {
        "role": "Bác sĩ",
        "message": "Chào em,\nDo cấu trúc da khác nhau ở mỗi vùng trên cơ thể cho nên da mặt, da tay và các bộ phận khác sẽ có mức độ tiết nhờn khác nhau.\nTiết bã nhờn ở tuổi dậy thì xảy ra khá nhiều, do nhiều nguyên nhân. Em chưa nói rõ em bị tiết nhờn cả mặt hay một vùng nào đó trên mặt? Em nên đến BS da liễu để BS xác định em thuộc loại da nhờn hay do viêm da khác.\nBiện pháp để giảm thiểu chất nhờn: em có thể dùng sữa rửa mặt dành cho da nhờn, giấy thấm dầu, thuốc bôi kiểm soát chất nhờn…"
    }]
]

In [42]:
l = len(conversation)

In [43]:
data_dict = {
    "source": ["Bệnh viện 115"] * l,
    "url": ["https://benhvien115.com.vn/tu-van-bac-si/ths-bs-huynh-thi-thuy-nga-giai-dap-thac-mac-ve-benh-da-lieu/2018032406345912"] * l,
    "conversation": conversation
}

save_csv(data_dict, "bv115_data_crawling.csv")

# VietSkin

In [5]:
conversation = [
    [{"role":"Người dùng","message": "Chào bác sĩ! Em mang thai được 3 tháng rồi, nghe các chị đồng nghiệp kể ai mang bầu cũng bị rạn da khiến em rất lo lắng, không biết khi nào mình bị rạn da. Có cách nào nhận biết triệu chứng của rạn da và làm thế nào để không bị rạn da ạ. Em cảm ơn!"},
     {"role":"Bác sĩ","message": "Chào em! Cảm ơn em vì đã quan tâm và gửi câu hỏi cho chuyên mục hỏi đáp bác sĩ da liễu.\nTheo thông tin em chia sẻ, rạn da khi mang bầu là hiện tượng thường gặp nhưng không phải tất cả các mẹ đều bị. Nguyên nhân là do cấu tạo biểu bì của từng người. Có người da có độ ẩm, độ đàn hồi tốt và ngược lại. Chính vì vậy, để không bị rạn da, em nên chủ động cung cấp độ ẩm cho làn da của mình ngay từ bây giờ. Bằng phương pháp sử dụng một số loại tinh dầu giúp dưỡng ẩm, bổ sung vitamin và dưỡng chất cho da, giúp da vùng bụng, ngực, đùi mềm mại và “co dãn” tốt hơn. \nMột số triệu chứng khi bị rạn da lúc mang bầu bạn nên chú ý: \nTùy cơ địa, cấu tạo da cũng như sự lên cân nhanh chậm của từng bà bầu mà hiện tượng rạn da có thể xuất hiện ở tháng thứ 4 đến tháng thứ 9 của thai kỳ. Trong khoảng thời gian này, cân nặng của bạn được tăng lên nhanh hơn 3 tháng đầu đồng nghĩa với việc thai nhi trong bụng lớn lên, khiến do da bụng của bạn sẽ bị rạn. Một số mẹ “chửa ngực” sẽ bị rạn “núi đôi”. Bên cạnh đó, có mẹ còn rạn đùi, mông, bắp chân gây mất thẩm mỹ. \nVết rạn ban đầu sẽ gây ngứa, có màu hơi đỏ hoặc hồng, tạo thành các vệt dọc, ngang ngoằn ngoèo trên da. Sau đó sẽ dần chuyển sang màu sẫm. Sau khi sinh xong lâu ngày sẽ có màu bạc, sờ thấy hơi nổi gờ hoặc lõm, dạng mấp mô lượn sóng, da vùng đó bị nhăn, nhìn rất xấu. \nVết rạn càng để lâu giống như sẹo, rất khó lành, chính vì vậy, bạn nên phòng rạn da trong khi mang thai càng sớm càng tốt. \nChúc bạn luôn khỏe đẹp!"}],
    [{"role":"Người dùng","message": "Em năm nay 15 tuổi và mới bị mụn cách đây 1 tuần, mụn bọc to và rất đau. Xin bác sĩ cho em biết cụ thể hơn về mụn trứng cá và tại sao mụn trứng cá hay xuất hiện ở lứa tuổi thanh thiếu niên ạ? Em cảm ơn bác sĩ."},
     {"role":"Bác sĩ","message": "Chào em! Cảm ơn em vì đã quan tâm và gửi câu hỏi cho chuyên mục hỏi đáp bác sĩ da liễu. \nTôi xin được giải đáp thắc mắc của em như sau: mụn trứng cá (bệnh trứng cá) biểu hiện trên làn da của em là hiện tượng của viêm nang lông tuyến bã. Tuyến bã là các tuyến nằm dưới da, cạnh lỗ chân lông, bài tiết chất nhờn giúp da mềm mại và có độ ẩm nhất định. Khi các tuyến này tiết nhiều, miệng tuyến bã bị bịt kín do sừng hóa, bụi bẩn… các chất nhờn sẽ tích tụ tạo thành nhân mụn. Trường hợp có sự tác động của các vi khuẩn như: P.acnes có sẵn trong nang lông tuyến bã tăng sinh, các tụ cầu, … Bội nhiễm từ ngoài gây nên hiện tượng mụn mủ, mụn bọc… Như em đang gặp phải. ngoài ra nếu không có hiện tượng bị nhiễm trùng thì sẽ tạo thành mụn đầu trắng, bị bịt kín. Khi mụn đầu trắng bị hở ra bên ngoài và gặp hiện tượng oxy hóa thì phần ngoài của nhân mụn sẽ trở thành màu đen, nên gọi là mụn đầu đen. \nMụn trứng cá thường khởi phát vào thời kỳ thanh thiếu niên là bởi vì ở độ tuổi này, cùng với sự phát triển nhanh về cơ thể, các tuyến mồ hôi và tuyến bã cũng phát triển và bài tiết mạnh hơn, nội tiết tố cũng được tăng cường sản xuất. kết hợp với tâm lý nóng vội, chăm sóc da không đúng cách khiến mụn càng nặng, lâu khỏi và dễ tái phát hơn. Vì vậy em nên thăm khám các y bác sĩ chuyên khoa để có hướng điều trị kịp thời, tránh để lâu gây viêm nhiễm trên diện rộng, mất thẩm mỹ và khó điều trị.\nChúc em thành công và có làn da như mong đợi!"}],
    [{"role":"Người dùng","message": "Da tay và da chân của tôi hiện nay bị bong da rất nhiều, bị khô rát mỗi khi tiếp xúc với xi măng. Trước kia tôi không bị như vậy, tôi mới bị cách đây vài tháng, đã đi khám ở viện da liễu thì họ bảo tôi bị viêm da cấp vậy xin hỏi tôi phải chữa như thế nào tôi thường xuyên tiếp xúc với xi măng, hiện tại da tay và da chân tôi rất khô rát và bị bong da rất nhiều, tôi muốn hỏi có bị ảnh hưởng gì không?"},
     {"role":"Bác sĩ","message": "Chào bác! Cảm ơn bác vì đã quan tâm và gửi câu hỏi cho chuyên mục hỏi đáp bác sĩ da liễu. \nTôi xin giải đáp thắc mắc của bác như sau: da có tác dụng là bảo vệ cơ thể khỏi các yếu tố gây hại từ môi trường, với hai hàng rào là lớp sừng và màng đáy. Nếu lớp sừng bị tổn thương thì sẽ làm tăng khả năng hấp thụ qua da, tăng mất nước qua lớp thượng bì. Khi đó hóa chất sẽ dễ dàng thấm qua da khiến chức năng bảo vệ và phục hồi của da sẽ dần bị cạn kiệt. kết quả không tránh khỏi là da bị viêm. \nTrong trường hợp này, bác có biểu hiện của bệnh viêm da do tiếp xúc với xi măng. \nBác nên đi kiểm tra dị ứng áp da, qua đó sẽ giúp bác tìm ra nguyên nhân gây viêm da, dị ứng da. đặc biệt, giúp cho những bệnh nhân bị viêm da cơ địa, sẩn ngứa, viêm da thần kinh… cải thiện được chất lượng cuộc sống. \nBác nên đến các trung tâm da liễu gần nhất làm xét nghiệm để bác sĩ cho bác đơn thuốc điều trị bệnh sớm khỏi chứ không nên tự ý sử dụng thuốc lung tung mà khiến tổn thương nặng hơn gây mãn tính khó chữa trị. Nếu bác còn điều gì thắc mắc hãy liên hệ trực tiếp với tôi qua phần mềm khám bệnh da liễu online vietskin, tôi sẽ trực tiếp thăm khám cho bác.\nChúc bác thành công!"}],
    [{"role":"Người dùng","message": "Em muốn hỏi tại sao phần trên mu của em lại nổi mụn đỏ, ngứa, đau như mụn trứng cá. Sau khi vỡ ra và thành sẹo vẫn không hết ngứa. Ban đầu chỉ có 1 mụn nhưng gần đây mọc lên nhiều hơn. Em rất lo lắng và khó chịu vì ngứa. Làm ơn cho em biết em bị bệnh gì và khám ở đâu, thời gian điều trị có lâu không ? Em xin cảm ơn bs."},
     {"role":"Bác sĩ","message": "Chào em! Cảm ơn em vì đã quan tâm và gửi câu hỏi cho chuyên mục hỏi đáp bác sĩ da liễu. \nTheo như em mô tả, có thể em đã bị viêm tuyến bã nang lông. \nNguyên nhân có thể do vệ sinh kém, do bệnh nhiễm trùng, hoặc cũng có thể do việc tẩy lông không đúng vệ sinh sẽ dễ dẫn đến em bị viêm nang lông. Ngoài ra những lúc có kinh nguyệt, dưới tác động của nội tiết tố, các tuyến bã của nang lông hoạt động mạnh nên hiện tượng viêm nhiễm nang lông cũng diễn ra nhiều hơn. \nNếu em viêm nang lông nhẹ, em chỉ cần giữ vệ sinh tốt vùng bị viêm và có thể dùng thêm các thuốc bôi ngoài da có chứa các chất kháng viêm như cortibion, betamethasone, fluocinolon … \nRất khó để nói nguyên nhân em bị mắc phải nếu không thăm khám và nói chuyện trực tiếp, em có thể đến các phòng khám chuyên khoa da liễu để được khám và điều trị đúng bệnh. \nChúc em mau khỏi!"}],
    [{"role":"Người dùng","message": "Bác sĩ tư vấn giúp em về 3 loại thuốc: Acyclovir (zovirax), famciclovir (famvir), valacyclovir (valtrex) dùng để chữa mụn rộp sinh dục. Nó là thuốc uống hay là dung dịch để bôi ạ? Em xin cảm ơn!"},
     {"role":"Bác sĩ","message": "Chào bạn! Cảm ơn bạn vì đã quan tâm và gửi câu hỏi cho chuyên mục hỏi đáp bác sĩ da liễu. \nĐối với thuốc trị mụn rộp sinh dục tôi xin được cung cấp 1 số thông tin về các loại thuốc trị mụn rộp mà bạn đã nêu như sau: \n- Acyclovir ( zovirax) \nAcyclovir là một đồng đẳng purine nucleoside tổng hợp với các tác động ức chế in vitro và in vivo chống lại virus gây bệnh herpes ở người. \nChỉ định: \nNhiễm virus herpes simplex da và niêm mạc, bao gồm herpes sinh dục khởi phát và tái phát. Ngăn chặn việc tái nhiễm herpes simplex trên người có miễn dịch bình thường. Dự phòng herpes simplex ở bệnh nhân suy giảm miễn dịch, bệnh zona (herpes zoster). các nghiên cứu cho thấy việc điều trị zona với zovirax trong giai đoạn sớm ảnh hưởng có lợi trên cảm giác đau và có thể làm giảm tần suất đau dây thần kinh sau zona (đau liên quan đến zona). \nDạng bào chế: có cả 2 dạng viên uống và thuốc bôi \n- Famciclovir (famvir) \nFamciclovir chịu sự biến đổi sinh học nhanh chóng thành hợp chất penciclovir có hoạt tính kháng virus, thuốc có hoạt tính ức chế chống lại virus herpes simplex type 1 (hsv-1), type 2 (hsv-2) và virus varicella zoster (vzv). \nChỉ định: \nĐiều trị nhiễm herpes zoster, nhiễm herpes simplex môi và sinh dục tái phát trên bệnh nhân nhiễm HIV \nDạng bào chế: thuốc uống \n- Valacyclovir (valtrex) \nValacyclovir là một loại thuốc kháng virus. Nó làm chậm sự phát triển và lây lan của virus herpes. Nó không chữa trị herpes, nhưng nó có thể làm giảm bớt các triệu chứng của nhiễm trùng. \nChỉ định: \nValacyclovir được sử dụng để điều trị các bệnh nhiễm trùng gây ra bởi virus herpes ở người lớn và trẻ em. Bệnh gây ra bởi virus herpes bao gồm herpes sinh dục, bệnh zona và thủy đậu. \nValacyclovir cũng có thể được sử dụng cho các mục đích khác không được liệt kê trong hướng dẫn thuốc, bạn nên tham khảo bác sĩ điều trị. \nDạng bào chế: thuốc uống. \nChúc bạn thành công!"}],
    [{"role":"Người dùng","message": "Chào bác sĩ, tôi là nữ, năm nay 25 tuổi. tôi hiện đang bị thủy đậu, có rất nhiều mụn phỏng dạng nước nổi dày đặc trên khắp người. bác sĩ cho tôi hỏi có nên chọc vỡ mụn nước này ra cho mau khỏi không? cảm ơn bác sĩ!"},
     {"role":"Bác sĩ","message": "Bạn bị thủy đậu và có rất nhiều mụn nước dày đặc trên khắp người. Tôi khuyên bạn không được làm vỡ những mụn thủy đậu này vì khi chọc vỡ mụn nước sẽ làm cho bệnh lây lan nhanh hơn và cũng làm tăng nguy cơ nhiễm khuẩn, bội nhiễm khó chữa trị hơn. Khi nốt thủy đậu bị nhiễm khuẩn thì dễ để lại sẹo sau khi khỏi bệnh. Do đó, bạn không nên làm vỡ mụn nước. Bạn nên khám bác sĩ để được hướng dẫn kê đơn chữa trị. \nBên cạnh đó, bạn có thể bôi xanh methylen lên các mụn khoảng 5 ngày, mỗi ngày 2 lần, sau các mụn sẽ teo lại. Các màng đó là màng chắn tự nhiên, chắn vi khuẩn và bụi bẩn. \nChúc bạn sức khỏe!"}]
]

In [6]:
l = len(conversation)

In [7]:
data_dict = {
    "source": ["VietSkin"] * l,
    "url": ["https://www.vietskin.vn/hoi-dap-bac-si-da-lieu-tu-van-tu-cac-chuyen-gia/"] * l,
    "conversation": conversation
}

save_csv(data_dict, "VietSkin_data_crawling.csv")

# Bệnh viện nhi đồng 2

In [52]:
urls = [
    "http://www.benhviennhi.org.vn/news/detail/3740/nhiem-trung-da.html",
    "http://www.benhviennhi.org.vn/news/detail/3719/cham-da.html",
    "http://www.benhviennhi.org.vn/news/detail/3695/ton-thuong-tren-da-be-co-nhieu-loai-khac-nhau-de-chan-doan-benh.html",
    "http://www.benhviennhi.org.vn/news/detail/3707/su-dung-cac-xa-phong-phu-hop-it-bi-di-ung-da.html",
    "http://www.benhviennhi.org.vn/news/detail/3694/co-can-cu-kieng-khi-be-bi-cham-sua.html",
    "http://www.benhviennhi.org.vn/news/detail/3693/cham-da-co-the-tai-di-tai-lai.html",
    "http://www.benhviennhi.org.vn/news/detail/3664/rom-say-thuong-xay-ra-vao-mua-nang-nong-.html",
    "http://www.benhviennhi.org.vn/news/detail/3662/noi-lam-tam-not-do-hai-ben-go-ma.html",
    "http://www.benhviennhi.org.vn/news/detail/3600/di-ung-anh-huong-den-suc-khoe-cua-be.html",
    "http://www.benhviennhi.org.vn/news/detail/3549/nhot-tren-dau-cua-be.html",
    "http://www.benhviennhi.org.vn/news/detail/3528/cham-sua.html",
    "http://www.benhviennhi.org.vn/news/detail/2772/khi-sot-be-thuong-noi-hong-ban.html",
    "http://www.benhviennhi.org.vn/news/detail/2689/be-bi-ham-va-co-dau-hieu-loet-o-nach.html",
    "http://www.benhviennhi.org.vn/news/detail/2684/be-hay-bi-di-ung-noi-me-day.html",
    "http://www.benhviennhi.org.vn/news/detail/2672/cham-la-loai-benh-co-the-tai-phat-va-thuong-khoi-benh-truoc-hai-thang-tuoi.html",
    "http://www.benhviennhi.org.vn/news/detail/2345/be-bi-ham-da-vung-co.html",
    "http://www.benhviennhi.org.vn/news/detail/2070/be-noi-nhieu-not-nho-ung-do-tren-mat--dieu-tri-mot-thang-chua-khoi.html",
    "http://www.benhviennhi.org.vn/news/detail/1442/sung-me-mong-chan-va-tay.html",
    "http://www.benhviennhi.org.vn/news/detail/516/lAc-sUa.html",
    "http://www.benhviennhi.org.vn/news/detail/1188/benh-choc-lo-ngoai-da-kha-pho-bien-o-tre-em-.html",
    "http://www.benhviennhi.org.vn/news/detail/1027/cham---benh-ngoai-da-thuong-gap-o-tre-em.html",
    "http://www.benhviennhi.org.vn/news/detail/2370/loet-luoi-va-chay-mau-chan-rang.html",
    "http://www.benhviennhi.org.vn/news/detail/2369/mun-trung-ca-o-tre-so-sinh.html",
    "http://www.benhviennhi.org.vn/news/detail/2329/Do-mo-hoi-trom.html",
    "http://www.benhviennhi.org.vn/news/detail/2322/da--noi-muc-nuoc-co-nhan-trang-duc.html",
    "http://www.benhviennhi.org.vn/news/detail/2319/viem-da-di-ung.html",
    "http://www.benhviennhi.org.vn/news/detail/2256/noi-me-day-la-hau-qua-cua-benh-di-ung.html",
    "http://www.benhviennhi.org.vn/news/detail/1554/tu-van-be-bi-den-long.html",
    "http://www.benhviennhi.org.vn/news/detail/1522/be-noi-hach-nhieu-kem-sot.html",
    "http://www.benhviennhi.org.vn/news/detail/1457/benh-cham-co-chung-ngua-duoc-khong.html",
    "http://www.benhviennhi.org.vn/news/detail/1427/benh-cham-co-the-chua-khoi-duoc-khong.html",
    "http://www.benhviennhi.org.vn/news/detail/527/viem-da-di-ung-o-tre.html",
    "http://www.benhviennhi.org.vn/news/detail/1183/luoi-be-co-nhieu-cham-trang-va-ngay-cang-to-.html",
    "http://www.benhviennhi.org.vn/news/detail/1176/be-hay-bi-rung-toc-.html",
    "http://www.benhviennhi.org.vn/news/detail/710/rom-say-co-the-tam-nuoc-la-tra-xanh-cho-be-duoc-khong-.html",
    "http://www.benhviennhi.org.vn/news/detail/713/noi-hot-cham-do-va-ngua---co-phai-be-mac-benh-phong-.html",
    "http://www.benhviennhi.org.vn/news/detail/709/lam-gi-khi-tre-bi-di-ung-voi-ta-giay-.html",
    "http://www.benhviennhi.org.vn/news/detail/674/khap-co-the-be-noi-nhung-hat-nho-gay-ngua-.html",
    "http://www.benhviennhi.org.vn/news/detail/673/khi-da-tre-bi-di-ung-voi-sua-.html",
    "http://www.benhviennhi.org.vn/news/detail/112/vung-da-chan-bi-noi-nhung-dom-do.html",
    "http://www.benhviennhi.org.vn/news/detail/113/be-noi-nhung-mun-nuoc-trang-o-cac-ngon-tay.html",
    "http://www.benhviennhi.org.vn/news/detail/401/vung-da-chan-bi-noi-nhung-dom-do.html",
    "http://www.benhviennhi.org.vn/news/detail/402/be-noi-nhung-mun-nuoc-trang-o-cac-ngon-tay.html",
    "http://www.benhviennhi.org.vn/news/detail/414/nhot-goc-ham.html",
    "http://www.benhviennhi.org.vn/news/detail/132/be-bi-viem-da-di-ung-tiet-ba.html",
    "http://www.benhviennhi.org.vn/news/detail/520/con-toi-bi-gi.html",
    "http://www.benhviennhi.org.vn/news/detail/541/sAn-ngUa-O-trE.html",
    "http://www.benhviennhi.org.vn/news/detail/559/trE-2-thAng-tuOi-bI-hAm-da.html",
    "http://www.benhviennhi.org.vn/news/detail/615/dI-Ung-da-O-trE-tU-2-3-thAng-tuOi.html",
    "http://www.benhviennhi.org.vn/news/detail/680/nhung-man-do-moc-duoi-co--tren-dau-va-lung.html"
]

data_dict = {
    "source": [],
    "url": [],
    "conversation": []
}

In [53]:
def benhviennd2_get_qa(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    es = soup.find_all('p')
    text = ""
    for e in es:
        if e.text.strip() not in text:
            text = text + e.text.strip() + "\n"

    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\n\s*\n+', '\n', text)
    
    index1 = text.find("Câu hỏi")
    index2 = text.find("Trả lời")
    index3 = text.find("Trả lời bởi")
    
    question = text[index1+8:index2].strip()
    answer = text[index2+8:index3].strip()
    
    return [{"role": "Người dùng", "message": question}, {"role": "Bác sĩ", "message": answer}]

In [54]:
def benhviennd2_crawling():
    for url in tqdm(urls, desc="Bệnh viện nhi đồng 2 - Crawling:"):
        html_content = get_html(url)
        conv = benhviennd2_get_qa(html_content)
        data_dict["source"].append("Bệnh viện nhi đồng 2")
        data_dict["url"].append(url)
        data_dict["conversation"].append(conv)
        save_csv(data_dict,"bvnd2_data_crawling.csv")
    print("Done!")

In [ ]:
benhviennd2_crawling()